In [10]:
import matplotlib.pyplot as plt
import numpy as np

In [11]:
c2 = 0.05
A = 3
B = 3
C = -3
tol = 1e-5
rtol = 1e-6
atol = 1e-12


In [12]:
def acc_y(x):
    return [[np.exp(np.sin(el ** 2)),
             np.exp(B * np.sin(el ** 2)),
             C * np.sin(el ** 2) + A,
             np.cos(el ** 2)] for el in x]


In [13]:
def f(x, y):
    return np.array([2 * x * np.sign(y[1]) * np.power(np.abs(y[1]), 1 / B) * y[3],
                     2 * B * x * np.exp(B * (y[2] - A) / C) * y[3],
                     2 * C * x * y[3],
                     - 2 * x * np.sign(y[0]) * np.log(np.abs(y[0]))])


In [14]:
def K(x, y, i, h, a, c):
    sum = 0
    for j in range(i):
        sum += a[i][j] * K(x, y, j, h, a, c)
    return f(x + c[i] * h, y + h * sum)


In [15]:
def app_y(x, y, h, a, c, b):
    sum = 0
    for i in range(2):
        sum += b[i] * K(x, y, i, h, a, c)
    return np.array([y + h * sum])

In [16]:
def rkutte(c2, h, x0=0, xn=5, y0=[1, 1, A, 1]):
    c = [0, c2]
    b = [0, 0]
    a = [[0, 0], [0, 0]]
    b[1] = 1 / (2 * c[1])
    b[0] = 1 - b[1]
    a[1][0] = c[1]
    x = list(np.arange(x0, xn, h))
    yj = list()
    yj.append(y0)
    yj = np.array(yj)
    for i in range(1, len(x)):
        yj = np.append(yj, app_y(x[i - 1], yj[i - 1], h, a, c, b), axis=0)
    return yj


Задание 3.1.

In [17]:
def task_1(h=0.001):
    x = list(np.arange(0, 5, h))
    plt.subplot(2, 1, 1)
    plt.title('c2 = 0.5')
    plt.plot(x, acc_y(x))
    plt.plot(x, rkutte(0.5, h))
    plt.subplot(2, 1, 2)
    plt.title('c2 = 0.05')
    plt.plot(x, rkutte(c2, h))
    plt.plot(x, acc_y(x))
    plt.show()

task_1()

/usr/local/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


Задание 3.2

In [18]:
xk = np.array([x for x in range(7)])
my = []
op = []
for k in xk:
    x = list(np.arange(0, 5, 1 / (2 ** k)))
    my_y = rkutte(c2, 1 / (2 ** k))
    op_y = rkutte(0.5, 1 / (2 ** k))
    real_y = acc_y(x)
    my.append(np.linalg.norm(np.abs(real_y[-1] - my_y[-1])))
    op.append(np.linalg.norm(np.abs(real_y[-1] - op_y[-1])))
plt.subplot(2, 1, 1)
plt.loglog(1 / (2 ** xk), my)
plt.subplot(2, 1, 2)
plt.loglog(1 / (2 ** xk), op)
plt.show()


Задание 3.3

In [23]:
def step(tol, x0, xn, y0=[1, 1, A, 1]):
    delt = (1 / max(np.abs(x0), np.abs(xn))) ** (2 + 1) + np.linalg.norm(f(x0, y0)) ** (2 + 1)
    h = (tol / delt) ** (1 / (2 + 1))
    y = y0 + h / 2 * (f(x0, y0) + f(x0 + h, y0 + h * f(x0, y0)))
    return h, y


In [24]:
h1, y1 = step(tol, 0, 5, y0=[1, 1, A, 1])
h = min(h1, step(tol, 0 + h1, 5, y1)[0])
print('start h =', h)
real_y = acc_y(x)
x = list(np.arange(0, 5, h))
y1 = rkutte(c2, h / 2)
opt = h * np.sqrt((tol / np.linalg.norm(np.abs(real_y[-1] - y1[-1]))))
print('optim h =', opt)
task_1(opt)


start h = 0.02247774790033101
optim h = 6.062312580613346e-05


In [28]:
def rkutte2(c2, h, x0=0,xn=5, y0=[1, 1, A, 1]):
    c = [0, c2]
    b = [0, 0]
    a = [[0, 0], [0, 0]]
    b[1] = 1 / (2 * c[1])
    b[0] = 1 - b[1]
    a[1][0] = c[1]
    x = [x0, x0 + h]
    yj = list()
    yj.append(y0)
    yj = np.array(yj)
    for i in range(1, len(x)):
        yj = np.append(yj, app_y(x[i - 1], yj[i - 1], h, a, c, b), axis=0)
    return yj


Задание 4

In [32]:
def optim(rtol, atol):
    y0 = np.array([1, 1, A, 1])
    x = [0]
    y = [y0]
    tol = rtol * np.linalg.norm(y0) + atol
    h1, u1 = step(tol, 0, 5, y0)
    h = min(h1, step(tol, 0 + h1, 5, u1)[0])
    h_1 = h
    h_max = h
    y1 = rkutte2(0.05, h, x[-1], x[-1] + 2 * h, y0)[-1]
    halfy = rkutte2(0.05, h / 2, x[-1], x[-1] + 3 * h / 2, y0)[-1]
    y2 = rkutte2(0.05, h / 2, x[-1], x[-1] + 3 * h / 2, halfy)[-1]
    r_1 = (y2 - y1) / (1 - 2 ** (-2))
    r_2 = (y2 - y1) / ((2 ** 2) - 1)
    while x[-1] < 5:
        if np.linalg.norm(r_1) > tol * 2 ** 2:
            h_1 = 0.5 * h_1
            y1 = halfy
            halfy = rkutte2(0.05, h_1 / 2, x[-1], x[-1] + 3 * h_1 / 2, y[-1])[-1]
            y2 = rkutte2(0.4, h_1 / 2, x[-1], x[-1] + 3 * h_1 / 2, halfy)[-1]
            r_1 = (y2 - y1) / (1 - 2 ** (-2))
            r_2 = (y2 - y1) / ((2 ** 2) - 1)
            continue
        elif tol < np.linalg.norm(r_1) <= tol * 2 ** 2:
            h_2 = h_1 / 2
            x.append(x[-1] + h_1)
            y.append(y2 + r_2)
            h_max = max(h_max, h_1)
        elif tol * (2 ** (-2 - 1)) <= np.linalg.norm(r_1) <= tol:
            h_2 = h_1
            x.append(x[-1] + h_1)
            y.append(y1 + r_1)
            h_max = max(h_max, h_1)
        elif np.linalg.norm(r_1) < tol * (2 ** (-2 - 1)):
            h_2 = min(2 * h_1, h_max)
            x.append(x[-1] + h_1)
            y.append(y1 + r_1)
            h_max = max(h_max, h_1)
        h_1 = h_2
        y1 = rkutte2(0.05, h_1, x[-1], x[-1] + 2 * h_1, y[-1])[-1]
        halfy = rkutte2(0.05, h_1 / 2, x[-1], x[-1] + 3 * h_1 / 2, y[-1])[-1]
        y2 = rkutte2(0.05, h_1 / 2, x[-1], x[-1] + 3 * h_1 / 2, halfy)[-1]
        r_1 = (y2 - y1) / (1 - 2 ** (-2))
        r_2 = (y2 - y1) / ((2 ** 2) - 1)
        tol = rtol * np.linalg.norm(y[-1]) + atol
    return np.array(x), np.array(y)
xopt, yopt = optim(rtol, atol)
plt.subplot(2, 1, 1)
plt.plot(xopt, yopt)
plt.subplot(2, 1, 2)
plt.plot(xopt, acc_y(xopt))
plt.show()
